# Spam real message classifier
###### simple probabilistic classification of spam and real messages using a basic word occurrence-based approach

In [48]:
import math  
import numpy as np  
from collections import Counter  
import pandas as pd  
import os

In [49]:
# Read the CSV file 
current_file = os.getcwd()
file_path = os.path.join(current_file, "spam.csv")
data = pd.read_csv(file_path, encoding = "ISO-8859-1")
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [50]:
# Select only the first two columns of the dataset
data = data.iloc[:, 0:2]

# Convert the 'v1' column to binary encoding (1 for 'ham', 0 for 'spam')
data["v1"] = data["v1"].apply(lambda x: 1 if x == "ham" else 0)

In [51]:

# Extract real and spam sentences from the dataset for training and testing
real_sentences = data[data["v1"] == 1]["v2"]
real_sentences_train = real_sentences[int(len(real_sentences)*0.3):]
real_sentences_test = real_sentences[:int(len(real_sentences)*0.3)]

spam_sentences = data[data["v1"] == 0]["v2"]
spam_sentences_train = spam_sentences[int(len(spam_sentences)*0.3):]
spam_sentences_test = spam_sentences[:int(len(spam_sentences)*0.3)]


In [52]:
# Split sentences into individual words
real_words = " ".join(real_sentences_train).split(" ")
spam_words = " ".join(spam_sentences_train).split(" ")

In [53]:
# Combine all words from real and spam sentences
unified_words = real_words + spam_words

# Find words that appear in both real and spam sentences
intersection_words = set(real_words).intersection(set(spam_words))

In [54]:
# Count occurrences of words in real and spam sentences
occurrences_real = dict(Counter(real_words))
occurrences_spam = dict(Counter(spam_words))

# Calculate probability of each word given that it is in a real or spam sentence
prob_of_w_given_real = {x: y / len(real_words) for x, y in occurrences_real.items()}
prob_of_w_given_spam = {x: y / len(spam_words) for x, y in occurrences_spam.items()}

In [55]:

# Calculate the prior probabilities of real and spam sentences
p_real = len(real_words) / (len(real_words) + len(spam_words))
p_spam = len(spam_words) / (len(real_words) + len(spam_words))


In [56]:
# Function to determine if a sentence is real or spam
def sentence_is_real(sentence):
    analyzed_words = [x for x in sentence.split(" ") if x in intersection_words]

    real_and_this_sentence = math.log(p_real)
    real_and_this_sentence += sum([np.log(prob_of_w_given_real[x]) for x in analyzed_words])

    spam_and_this_sentence = math.log(p_spam)
    spam_and_this_sentence += sum([np.log(prob_of_w_given_spam[x]) for x in analyzed_words])

    return real_and_this_sentence > spam_and_this_sentence

In [57]:
# Calculate accuracy for real sentences
quantity = 0
for x in real_sentences_test:
    if sentence_is_real(x):
        quantity += 1
print(f"{quantity / len(real_sentences_test)} of words are correctly predicted as real")

0.9571527297857636 of words are correctly predicted as real


In [58]:
# Calculate accuracy for spam sentences
quantity = 0
for x in spam_sentences_test:
    if not sentence_is_real(x):
        quantity += 1
print(f"{quantity / len(spam_sentences_test)} of words are correctly predicted as spam")


0.8973214285714286 of words are correctly predicted as spam
